In [465]:
import pandas as pd

In [466]:
csv_1 = pd.read_csv('if_else(1).csv', index_col=0)
csv_2 = pd.read_csv('if_else(2).csv', index_col=0)

In [467]:
mask = csv_1.index.isin(csv_2.index)
csv_1.loc[mask, 'rationales'] = csv_2.loc[csv_2.index.isin(csv_1.index), 'rationales']

In [468]:
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...
...,...,...,...
5587,A woman is tugging on a white sheet and laughing,The woman is changing the sheets on her bed.,8. Premise: A woman is tugging on a white shee...
5588,A man is looking over a small cliff.,A woman is looking,9. Premise: A man is looking over a small clif...
5589,A man in goggle swimming.,A man swimming in the water with goggles.,10. Premise: A man in goggles swimming.\n H...
5590,Two men are standing in front of musical mixin...,There are people indoors.,"The correct answer is ""entailment."" The premi..."


In [469]:
csv_3 = pd.read_csv('if_else(3).csv', index_col=0)
csv_4 = pd.read_csv('if_else(4).csv', index_col=0)
csv_1 = pd.concat([csv_1, csv_3], axis=0)
csv_1 = pd.concat([csv_1, csv_4], axis=0)
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...
...,...,...,...
127,An older Asian man is baking bread sticks in a...,A man is baking a cake.,8. Premise: An older Asian man is baking bread...
128,"On a rainy day, a woman holds an umbrella abov...",A woman is walking down the beach in the sunsh...,"9. Premise: On a rainy day, a woman holds an u..."
129,A soccer goalie catching the ball as it tries ...,A punter kicks the football down the field.,10. Premise: A soccer goalie catching the ball...
130,A man with a blue hat and a cane is sitting on...,A hatless mam sitting on a curb.,"The correct answer is ""Contradiction."" Reason..."


In [470]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [471]:
prompt_template= '''
Premise: %s, 
Hypothesis: %s.
Given the premise '%s' and hypothesis %s, which among the choices "entailment", "neutral", "contradiction", is the correct answer? Explain your reasoning using conditional statements.\n
'''

In [472]:
error = csv_1[csv_1.rationales.isna()]

In [473]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)

In [474]:
error

,premise,hypothesis,rationales,prompt


In [475]:
# csv_1['label'] = train_df['label']

In [476]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [477]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [478]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)

In [479]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [480]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,rationales,split,correct_index,LLM_answer


In [481]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [482]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [483]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [484]:
csv_1[csv_1.index == 2]

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,


In [485]:
csv_1[csv_1['premise'] == 'Two male dancers dressed in blue shirts and black pants perform a routine in front of six women sitting beside them.']

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
Two male dancers dressed in blue shirts and black pants perform a routine in front of six women sitting beside them.</s>Nobody has a shirt.,Two male dancers dressed in blue shirts and bl...,Nobody has a shirt.,1. Premise: Two male dancers dressed in blue s...,1. premise: two male dancers dressed in blue s...,"{'contradiction': 334, 'contradicts': 191}",contradicts
Two male dancers dressed in blue shirts and black pants perform a routine in front of six women sitting beside them.</s>Nobody has a shirt.,Two male dancers dressed in blue shirts and bl...,Nobody has a shirt.,5. Premise: Two male dancers dressed in blue s...,5. premise: two male dancers dressed in blue s...,"{'contradiction': 309, 'contradicts': 180}",contradicts
Two male dancers dressed in blue shirts and black pants perform a routine in front of six women sitting beside them.</s>Nobody has a shirt.,Two male dancers dressed in blue shirts and bl...,Nobody has a shirt.,3. Premise: Two male dancers dressed in blue s...,3. premise: two male dancers dressed in blue s...,"{'contradiction': 456, 'contradicts': 299}",contradicts


In [486]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [487]:
paper.shape

(5592, 6)

In [554]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'neutral': 467},neutral
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'neutral': 315},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'entailment': 356},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entailment': 423},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'neutral': 354},neutral
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,2. Premise: Three women walk down a city stree...,2. premise: three women walk down a city stree...,{'neutral': 410},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,7. Premise: A basketball player slam dunks the...,7. premise: a basketball player slam dunks the...,"{'neutral': 192, 'contradiction': 419}",neutral
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,2. Premise: Two kids wearing white robes are p...,2. premise: two kids wearing white robes are p...,"{'entailment': 495, 'entails': 456}",entails


In [561]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'

In [567]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'neutral': 467},neutral
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'neutral': 315},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'entailment': 356},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entailment': 423},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'neutral': 354},neutral
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,2. Premise: Three women walk down a city stree...,2. premise: three women walk down a city stree...,{'neutral': 410},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,7. Premise: A basketball player slam dunks the...,7. premise: a basketball player slam dunks the...,"{'neutral': 192, 'contradiction': 419}",neutral
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,2. Premise: Two kids wearing white robes are p...,2. premise: two kids wearing white robes are p...,"{'entailment': 495, 'entails': 456}",entailment


In [563]:
paper['my_label'] = total_df['LLM_answer']

In [564]:
sum(paper.label == paper.my_label) / len(paper)

0.6114091559370529

In [565]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [566]:
total_df.to_csv('if_else - full.csv')

In [ ]:
csv_1.to_csv('if_else - full.csv', index=False)

In [ ]:
pd.read_csv('if_else(4).csv', index_col=0, delimiter=',')

,premise,hypothesis,rationales,Unnamed: 4
0,Two topless men and a woman wearing a dress ar...,the trees in the background have no leaves an...,People are in water.,1. Premise: Two topless men and a woman wearin...
1,Three women walk down a city street,one looks inside her handbag as she walks.,Three women chat at home.,2. Premise: Three women walk down a city stree...
2,Two male dancers dressed in blue shirts and bl...,Nobody has a shirt.,3. Premise: Two male dancers dressed in blue s...,NaN
3,"Two men and a woman in shorts and t-shirts, ca...",They are all women.,4. Premise: Two men and a woman in shorts and ...,NaN
4,Two women in sunglasses are crossing a street ...,Nobody has sunglasses.,5. Premise: Two women in sunglasses are crossi...,NaN
...,...,...,...,...
127,An older Asian man is baking bread sticks in a...,A man is baking a cake.,8. Premise: An older Asian man is baking bread...,NaN
128,"On a rainy day, a woman holds an umbrella abov...",A woman is walking down the beach in the sunsh...,"9. Premise: On a rainy day, a woman holds an u...",NaN
129,A soccer goalie catching the ball as it tries ...,A punter kicks the football down the field.,10. Premise: A soccer goalie catching the ball...,NaN
130,A man with a blue hat and a cane is sitting on...,A hatless mam sitting on a curb.,"The correct answer is ""Contradiction."" Reason...",NaN
